NFL season is upon us so let's engage in the time honored ritual of fatuously trying to predict games that a coin flip would do better at predicting. 

In this blog post we will:
* get started with nfldb (limitations as well)
* query our data
* format for processing
* deploy logistic regression

# nfldb

some words on nfldb and its limitations

# Querying the Data

now we have postresql set up we can query the data

In [9]:
%matplotlib inline
import psycopg2
conn=psycopg2.connect("dbname='nfldb' user='kthomas1' host='localhost' password='' port=5432")
cur=conn.cursor()

# our first query to get the most common colleges for nfl players
cur.execute("""SELECT college, count(*) as freq 
from player 
where college is not null 
group by college 
order by freq DESC 
limit 10""")


In [10]:
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

college_count = cur.fetchall()
college_count_head = [("School","Count")] + college_count
college_count_table = ff.create_table(college_count_head)
iplot(college_count_table, filename='college_count_table')

In [11]:
conn=psycopg2.connect("dbname='nfldb' user='kthomas1' host='localhost' password='' port=5432")
cur=conn.cursor()

cur.execute("""SELECT full_name, position, uniform_number, height, weight, college
from player
where team='CAR'
limit 10""")

car_roster = cur.fetchall()

#here we just display the raw results without
car_roster

[('Robert McClain', 'DB', 27, 69, 195, 'Connecticut'),
 ('Daryl Williams', 'OT', 60, 78, 335, 'Oklahoma'),
 ('Jonathan Stewart', 'RB', 28, 70, 240, 'Oregon'),
 ('Brenton Bersin', 'WR', 11, 75, 210, 'Wofford'),
 ('Ted Ginn', 'WR', 19, 71, 180, 'Ohio State'),
 ('James Bradberry', 'CB', 24, 73, 210, 'Samford'),
 ('Daryl Worley', 'CB', 26, 73, 205, 'West Virginia'),
 ('Ryan Kalil', 'C', 67, 74, 300, 'USC'),
 ('Dean Marlowe', 'FS', 29, 73, 210, 'James Madison'),
 ('Tyler Larsen', 'OG', 69, 76, 335, 'Utah')]

problems with the dataset as with players not on team

In [12]:
# import pandas and numpy for 
import pandas as pd
import numpy as np

# query game results
conn=psycopg2.connect("dbname='nfldb' user='kthomas1' host='localhost' password='' port=5432")
cur=conn.cursor()
cur.execute("""select season_year, week, home_team, home_score, away_team, away_score
from game
where season_type='Regular'""")

#put results into a dataframe
raw = cur.fetchall()
game_results=pd.DataFrame(raw,columns=['year','week','home_team','home_score','away_team','away_score'])

#replace la with stl
game_results.replace(to_replace='LA', value='STL', inplace=True)

# compute wins and ties
game_results['home_win'] = [1 if x>y else 0 for x,y in zip(game_results['home_score'],game_results['away_score'])]
game_results['away_win'] = [1 if x<y else 0 for x,y in zip(game_results['home_score'],game_results['away_score'])]
game_results['tie'] = [1 if x==y else 0 for x,y in zip(game_results['home_score'],game_results['away_score'])]

# sort the dataframe
game_results=game_results.sort_values(by=['year','home_team','week'])

# add week 2017 to results --> not yet in nfldb so has to be added manually
w2017 = pd.read_csv('~/desktop/2017_w1.csv')
game_results = game_results.append(w2017)

game_results.head(10)


,away_score,away_team,away_win,home_score,home_team,home_win,tie,week,year
864,20,SF,1,16,ARI,0,0,1,2009
153,31,IND,1,10,ARI,0,0,3,2009
250,21,HOU,0,28,ARI,1,0,5,2009
727,34,CAR,1,21,ARI,0,0,8,2009
281,20,SEA,0,31,ARI,1,0,10,2009
829,17,MIN,0,30,ARI,1,0,13,2009
917,10,STL,0,31,ARI,1,0,16,2009
361,33,GB,1,7,ARI,0,0,17,2009
860,7,MIA,0,19,ATL,1,0,1,2009
1115,20,CAR,0,28,ATL,1,0,2,2009


explain list comprehension used to code wins/ties

In [13]:
#base line is just always guessing home wins --> compare to cross validated accuracy score
total_games = len(game_results)
home_wins = game_results.home_win.sum()
home_win_rate = home_wins/total_games
#print as a nice string

In [14]:
cur.execute("""select drive.pos_team, drive.drive_id, drive.pos_time, drive.first_downs, drive.yards_gained, drive.play_count, drive.result, game.season_year, game.week, game.season_type, game.home_team, game.away_team
from drive
inner join game on drive.gsis_id=game.gsis_id
where season_type='Regular'""")

stats = cur.fetchall()

stats = pd.DataFrame(stats,columns=['pos_team','drive_id','pos_time','first_downs','yards_gained','play_count','result','season_year','week','season_type','home_team','away_team'])

#replace la with stl
stats.replace(to_replace='LA', value='STL', inplace=True)

# encode points results
stats['points'] = [3 if x=="Field Goal" else 7 if x=="Touchdown" else 0 for x in stats['result']]

# encode turnover results
stats['turnover'] = [1 if x==("Interception" or "Fumble" or "Safety" or "Blocked FG" or "Fumble, Safety" or "Blocked Punt" or "Blocked Punt, Downs" or "Blocked FG, Downs") else 0 for x in stats['result']]


linking the two datasets
fixing LA and STL 
assumptions on turnovers -- not precise but will be fine for now

In [15]:
# defense
stats_d = stats
stats_d['opp_team'] = np.where(stats_d['pos_team']==stats_d['home_team'], stats_d['away_team'], stats_d['home_team'])
#subset to defensive stats
stats_d = stats_d[['season_year','week','opp_team','yards_gained','points','turnover']]
# rename columns
stats_d.columns = ['year','week','team','yards_allowed','points_allowed','turnovers_forced']

stats_d_2017 = pd.read_csv('~/desktop/nfl_2017_stats_d.csv')
stats_d=stats_d.append(stats_d_2017)

# look at the numbers
# table = pd.pivot_table(stats_d, values=['yards_allowed','points_allowed','turnovers_forced'], index=['year', 'team'], aggfunc=np.sum)
# table.sort_values(by='turnovers_forced',ascending=False)


note the [[ notation for subsetting
renaming columns
manually adding in 2017 statistics (needs imporovement)

comment on the table

now need for aggregate rolling 5 week numbers

In [16]:
# aggregate rolling 5 week
## sort at year, team, week
stats_d.sort_values(by=['team','year','week'],inplace=True)
## sum across year team week
stats_d=stats_d.groupby(by=['team','year','week'],as_index=False).sum()
## rolling 2 week lagged
rolling = stats_d.groupby(['team'],as_index=False)['yards_allowed','points_allowed','turnovers_forced'].rolling(5).sum().shift(1).reset_index()
## join together
stats_d=stats_d.join(rolling,lsuffix='_weekly',rsuffix='_rolling')

rolling five weeks
sort; collapse to team-year-week level; 
rolling -> explain rolling, shift and reset_index
join rolling stats with weekly stats - explain suffix

In [17]:
# offense
stats_o = stats
stats_o=stats_o.rename(columns = {'pos_team':'team'})
stats_o=stats_o.rename(columns = {'season_year':'year'})
stats_o = stats_o[['team','year','week','first_downs','yards_gained','play_count','points','turnover']]

stats_o_2017 = pd.read_csv('~/desktop/nfl_2017_stats_o.csv')
stats_o=stats_o.append(stats_o_2017)

# aggregate rolling 5 week
## sort at year, team, week
stats_o.sort_values(by=['team','year','week'],inplace=True)
## sum across year team week
stats_o=stats_o.groupby(by=['team','year','week'],as_index=False).sum()
## rolling 5 week lagged
rolling = stats_o.groupby(['team'],as_index=False)['first_downs','yards_gained','play_count','points','turnover'].rolling(5).sum().shift(1).reset_index()
## join together
stats_o=stats_o.join(rolling,lsuffix='_weekly',rsuffix='_rolling')

In [18]:
## combine offense and defense
stats_o = stats_o.drop(['level_0','level_1'], axis=1)
stats_d = stats_d.drop(['level_0','level_1'], axis=1)
stats_od=pd.concat([stats_d,stats_o],axis=1)
stats_od=stats_od.T.drop_duplicates().T


#simplify dataset
stats_od=stats_od[stats_od['year']!=2009]
weekly_stats = [col for col in stats_od if col.endswith('weekly')]
stats_od = stats_od.drop(weekly_stats, axis=1)

# convert to numeric
stats_od=stats_od.apply(pd.to_numeric, errors='ignore')


In [19]:
games = game_results

stats_od.columns=['team','year','week','ya','pa','tf','fd','yg','pc','p','t']

games=pd.merge(pd.merge(games,stats_od,left_on=['home_team','year','week'],right_on=['team','year','week']),stats_od,left_on=['away_team','year','week'],right_on=['team','year','week'],suffixes=['_home','_away'])



diffs=['ya','pa','tf','fd','yg','pc','p','t']

for i in diffs:
    diff_column = i + "_diff"
    home_column = i + "_home"
    away_column = i + "_away"
    games[diff_column] = games[home_column] - games[away_column]


home = [col for col in games if col.endswith('home')]
away = [col for col in games if col.endswith('away')]

games_w = games.drop(home,axis=1)
games_w = games_w.drop(away,axis=1)

# Model

In [20]:
import statsmodels.api as sm

games_16 = games_w[games_w['year']<2017] 
games_17 = games_w[games_w['year']==2017]

games_16['intercept'] = 1.0
games_17['intercept'] = 1.0

training_cols = [col for col in games_w if col.endswith('diff')]
training_cols = training_cols + ["intercept"]

logit = sm.Logit(games_16['home_win'], games_16[training_cols])

result = logit.fit()

print(result.summary())


Optimization terminated successfully.
         Current function value: 0.644531
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               home_win   No. Observations:                 1792
Model:                          Logit   Df Residuals:                     1783
Method:                           MLE   Df Model:                            8
Date:                Thu, 31 Aug 2017   Pseudo R-squ.:                 0.05770
Time:                        20:39:17   Log-Likelihood:                -1155.0
converged:                       True   LL-Null:                       -1225.7
                                        LLR p-value:                 1.179e-26
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
ya_diff       -0.0003      0.000     -1.050      0.294        -0.001     0.000
pa_diff       -0.0061      0.

/Users/kthomas1/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/kthomas1/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [21]:
#log odds
print(np.exp(result.params))

ya_diff      0.999744
pa_diff      0.993909
tf_diff      1.006948
fd_diff      1.010130
yg_diff      1.000156
pc_diff      0.995846
p_diff       1.009127
t_diff       0.974167
intercept    1.382711
dtype: float64


In [22]:
preds=result.predict(games_17[training_cols])

games_17['win_prob'] = preds

games_17['winner'] = np.where(games_17['win_prob']>.5,games_17['home_team'],games_17['away_team'])

games_17[['home_team','away_team','winner','win_prob']]

/Users/kthomas1/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/kthomas1/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,home_team,away_team,winner,win_prob
1792,NE,KC,NE,0.710694
1793,DET,ARZ,ARZ,0.476726
1794,CHI,ATL,ATL,0.294886
1795,CIN,BAL,CIN,0.668382
1796,HOU,JAC,HOU,0.608525
1797,BUF,NYJ,BUF,0.727418
1798,TEN,OAK,TEN,0.530880
1799,WAS,PHL,WAS,0.668242
1800,CLE,PIT,PIT,0.413322
1801,MIA,TB,TB,0.496940


## Better Model

normalize variables
cross validation
shrinkage

do same thing but with sklearn just to verify

In [23]:
from sklearn.linear_model import LogisticRegression

games_16 = games_w[games_w['year']<2017] 
games_17 = games_w[games_w['year']==2017]

training_cols = [col for col in games_w if col.endswith('diff')]

logit = LogisticRegression(fit_intercept=True)
logit.fit(games_16[training_cols],games_16['home_win'])

preds=logit.predict(games_17[training_cols])

games_17['prediction'] = preds

games_17['winner'] = np.where(games_17['prediction']==1,games_17['home_team'],games_17['away_team'])

games_17['win_prob'] = logit.predict_proba(games_17[training_cols])[:,1]

games_17[['home_team','away_team','winner','win_prob']]

/Users/kthomas1/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/kthomas1/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/kthomas1/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

,home_team,away_team,winner,win_prob
1792,NE,KC,NE,0.710502
1793,DET,ARZ,ARZ,0.476534
1794,CHI,ATL,ATL,0.294762
1795,CIN,BAL,CIN,0.668178
1796,HOU,JAC,HOU,0.608317
1797,BUF,NYJ,BUF,0.727229
1798,TEN,OAK,TEN,0.530688
1799,WAS,PHL,WAS,0.668059
1800,CLE,PIT,PIT,0.413144
1801,MIA,TB,TB,0.496771


why are the results a little different --> sklearn defaults to using the l2 penalty; see the get_params() below

get accuracy from above

cross validate

In [24]:
logit.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [193]:
from sklearn import preprocessing
from sklearn import metrics, cross_validation

training_cols = [col for col in games_w if col.endswith('diff')]
g16_scaled = pd.DataFrame(preprocessing.scale(games_16[training_cols]))


scores = cross_validation.cross_val_score(logit, g16_scaled, games_16['home_win'], cv=10)
# accuracy
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.62 (+/- 0.07)
